In [1]:
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/Users/viviensiew/.local/share/virtualenvs/llm-zoomcamp-0DI9tvdQ/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

### Load Embedding Model
Load the model and perform a quick test with sample text.

In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [4]:
v = model.encode('I just discovered the course. Can I still join?')
len(v)

384

Example of vector dot product with itself. It should produce highest result (~1) due to similarity (exact match).

In [5]:
v.dot(v)

1.0

### Create the 3 vectors

Create a question vector, answer vector and question_answer vector for comparison. Note that the vector dimensions must match the dimension of the embedding model's output.

In [6]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|████████████████████████████████████████████████████████████████████████| 948/948 [03:30<00:00,  4.51it/s]


### Index the encoded document in ES index

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████| 948/948 [00:07<00:00, 121.74it/s]


### Encode a query

In [9]:
query = 'I just discovered the course. Can I still join it?'

In [10]:
v_q = model.encode(query)

### Setup ES Search with Vector Search

The vector search is done using K-Nearest Neighbours (KNN) where the search field is dynamic since we are evaluating 3 methods:
* Embedded questions in FAQ.
* Embedded answers in FAQ.
* Embedded questions and answers in FAQ.

In [11]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

### Method 1: Vector Search based on embedded question only
For each record in this data, we parse in the encoded question, specify the vector search field = embedded question (from FAQ not ground-truth data) and the relevant course for filtering.

In [12]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

### Load ground truth data

In [13]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [14]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [15]:
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

### Setup evaluation metrics: Hit Rate and Mean Reciprocal Rank (MRR)

In [16]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [17]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

For each question in ground-truth data, parse to ES Search to get back 5 results, which are then evaluated with HR and MRR.

In [18]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [19]:
evaluate(ground_truth, question_vector_knn)

100%|██████████████████████████████████████████████████████████████████████| 4627/4627 [02:04<00:00, 37.25it/s]


{'hit_rate': 0.773071104387292, 'mrr': 0.6666810748505158}

### Method 2: Vector Search based on embedded answers only
For each record in this data, we parse in the encoded question, specify the vector search field = embedded answer and the relevant course for filtering.

In [20]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [21]:
evaluate(ground_truth, text_vector_knn)

100%|██████████████████████████████████████████████████████████████████████| 4627/4627 [01:55<00:00, 40.23it/s]


{'hit_rate': 0.8286146531229739, 'mrr': 0.7062315395144454}

### Method 3: Vector Search based on embedded questions and answers
For each record in this data, we parse in the encoded question, specify the vector search field = embedded question and answer, and the relevant course for filtering.

In [22]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

evaluate(ground_truth, question_text_vector_knn)

100%|██████████████████████████████████████████████████████████████████████| 4627/4627 [01:54<00:00, 40.42it/s]


{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}

### KNN with 3 dense vectors
The ES Search's KNN param only accepts 1 field of vector embeddings as search field, hence this is ChatGPT's suggestion on how to use more than one vector fields in search. See "script". 

In [23]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [24]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

evaluate(ground_truth, vector_combined_knn)

100%|██████████████████████████████████████████████████████████████████████| 4627/4627 [01:52<00:00, 41.00it/s]


{'hit_rate': 0.9023125135076724, 'mrr': 0.804480945176861}